In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
from colorama import Fore, Style

In [ ]:
elig_df = pd.read_excel ('..\Data\TEIS-NSS Project Data 10-2022.xlsx', sheet_name = 'Elig Timeline Rpt 2018-2022') 

In [ ]:
elig_df['Exit Reason'].value_counts()

In [ ]:
DE = elig_df[elig_df['Init. Elig. Category'] == 'Developmental Evaluation']
DE_count = pd.DataFrame(DE['Exit Reason'].value_counts())

DC = elig_df[elig_df['Init. Elig. Category'] == 'Diagnosed Condition']
DC_count = pd.DataFrame(DC['Exit Reason'].value_counts())

DC_DE = elig_df[elig_df['Init. Elig. Category'] == 'Diagnosed Condition, Developmental Evaluation']
DC_DE_count = pd.DataFrame(DC_DE['Exit Reason'].value_counts())

In [ ]:
null = elig_df['Exit Reason'].isnull().sum()
total_rows = elig_df['Exit Reason'].value_counts().sum()
DEcount = DE['Exit Reason'].value_counts().sum()
DCcount = DC['Exit Reason'].value_counts().sum()
DCDEcount = DC_DE['Exit Reason'].value_counts().sum()

In [ ]:
print('\33[1mDevelopment Evaluation Count: \33[m' + str(DEcount))
print('\33[1mDiagnosed Condition Count: \33[m' + str(DCcount))
print('\33[1mBoth (DE & DC) Count: \33[m' + str(DCDEcount))
print('\33[1mNull Count: \33[m' + str(null))
num = [DEcount, DCcount, DCDEcount, null]
print('\33[1mTotal Count: \33[m' + str(sum(num)))
print('\33[1mTotal Row Count: \33[m' + str(total_rows))
print(Fore.RED + '\33[1mDifference: \33[m' + str(total_rows - sum(num)))

In [ ]:
DE_count = pd.DataFrame(DE_count.reset_index())
DE_count.rename (columns = {'index':'Exit Reason: Description','Exit Reason':'Exit Reason: Count'}, inplace=True)
DE_count['Exit Reason: Description'] = DE_count['Exit Reason: Description'].str.strip('618 - ')

DC_count = pd.DataFrame(DC_count.reset_index())
DC_count.rename (columns = {'index':'Exit Reason: Description','Exit Reason':'Exit Reason: Count'}, inplace=True)
DC_count['Exit Reason: Description'] = DC_count['Exit Reason: Description'].str.strip('618 - ')

DC_DE_count = pd.DataFrame(DC_DE_count.reset_index())
DC_DE_count.rename (columns = {'index':'Exit Reason: Description','Exit Reason':'Exit Reason: Count'}, inplace=True)
DC_DE_count['Exit Reason: Description'] = DC_DE_count['Exit Reason: Description'].str.strip('618 - ')

In [ ]:
merged = pd.merge(DC_DE_count, DC_count, how = "outer", on = 'Exit Reason: Description')
all_cat = pd.merge(DE_count, merged, how = "outer", on = 'Exit Reason: Description')
all_cat.rename(columns = {'Exit Reason: Description':'Exit Reason','Exit Reason: Count':'Development Evaluation','Exit Reason: Count_x':'Both', 'Exit Reason: Count_y': 'Diagnosed Condition'}, inplace=True)

In [ ]:
all_cat = all_cat.fillna(0)
all_cat['Both'] = all_cat['Both'].astype(int)
all_cat['Diagnosed Condition'] = all_cat['Diagnosed Condition'].astype(int)
all_cat = pd.DataFrame(all_cat)
all_cat.info()

In [ ]:
#all_cat['Exit Reason'] = all_cat['Exit Reason'].str.strip(' – Exit with no referrals' + 'Not eligible for' + 'eligibility not determined' + '- Exit with referral to other program') 
all_cat.describe()

In [ ]:
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Part B eligibility not determined', 'Aged Out')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Part B eligible', 'Aged Out')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Not eligible for Part B – Exit with no referrals', 'Aged Out')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Not eligible for Part B – Exit with referral to other programs', 'Aged Out')

In [ ]:
all_cat

In [ ]:
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Parent withdraw', 'Other')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Attempts to contact unsuccessful', 'Other')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Parent decline', 'Other')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Unable to contact', 'Other')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Moved out of state', 'Other')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('No meeting occurred', 'Other')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Completion of IFSP/ no longer requires services', 'Other')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Deceased', 'Other')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Referral less than 45 days', 'Other')
all_cat['Exit Reason'] = all_cat['Exit Reason'].replace('Screening – No action is required', 'Other')

In [ ]:
group = all_cat.groupby(by='Exit Reason').sum()

In [ ]:
df_group = pd.DataFrame(group.reset_index())
df_group

In [ ]:
df_group.describe()

In [ ]:
#Development Evaluation
df_group.groupby(['Exit Reason']).sum().plot(kind='pie', y='Development Evaluation', labels = ('',''), colors = ('gold','lemonchiffon'), fontsize = 15, autopct='%1.1f%%')
labels = ['Aged Out', 'Other']
plt.legend(labels, bbox_to_anchor=(.68,-.01),loc = 'lower right', fontsize=15, bbox_transform=plt.gcf().transFigure)
plt.title("Development Evaluation", fontsize = 15, fontweight = 'bold')
plt.ylabel('')

#Both
df_group.groupby(['Exit Reason']).sum().plot(kind='pie', y='Both', labels = ('',''), colors = ('orange','moccasin'), fontsize = 15, autopct='%1.1f%%')
labels = ['Aged Out', 'Other']
plt.legend(labels, bbox_to_anchor=(.68,-.01),loc = 'lower right', fontsize=15, bbox_transform=plt.gcf().transFigure)
plt.title("Both", fontsize = 15, fontweight = 'bold')
plt.ylabel('')

#Diagnosed Condition 
df_group.groupby(['Exit Reason']).sum().plot(kind='pie', y='Diagnosed Condition', labels = ('',''), colors = ('indianred','mistyrose'),fontsize = 15, autopct='%1.1f%%')
labels = ['Aged Out', 'Other']
plt.legend(labels, bbox_to_anchor=(.68,-.01),loc = 'lower right', fontsize=15, bbox_transform=plt.gcf().transFigure)
plt.title("Diagnosed Condition", fontsize = 15, fontweight = 'bold')
plt.ylabel('');